<div style="text-align: right" align="right"><i>Wesley Lau<br>December 1–25, 2022</i></div>

# Advent of Code 2022



# Day 0: Preparations

Imports from past AoC years:

In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import matplotlib.pyplot as plt
import re

Each day's work will consist of three tasks, denoted by three bulleted section:
- **Input**: Parse the day's input file.  I will  use the function `parse(day, parser, sep)`, which:
   - Reads the input file for `day`.
   - Prints out the first few lines of the file (to remind me, and the notebook reader, what's in the file).
   - Breaks the file into a sequence of *entries* separated by `sep` (default newline).
   - Applies `parser` to each entry and returns the results as a tuple.
       - Useful parser functions include `ints`, `digits`, `atoms`, `words`, and the built-ins `int` and `str`.
- **Part 1**: Understand the day's instructions and:
   - Write code to compute the answer to Part 1.
   - Record the answer with the `answer` function, which also serves as a unit test when the notebook is re-run.
- **Part 2**: Understand the second part of the instructions and:
   - Write code and record `answer` for Part 2.
   
Occasionally I'll introduce a **Part 3** where I explore beyond the instructions.

Here are the helper functions for `answer` and `parse`:

In [2]:
def answer(puzzle_number, got, expected) -> bool:
    """Verify the answer we got was the expected answer."""
    assert got == expected, f'For {puzzle_number}, expected {expected} but got {got}.'
    return True

def parse(day, parser=str, sep='\n', print_lines=7) -> tuple:
    """Split the day's input file into entries separated by `sep`, and apply `parser` to each."""
    fname = f'AOC2022/input{day}.txt'
    text  = open(fname).read()
    entries = mapt(parser, text.rstrip().split(sep))
    if print_lines:
        all_lines = text.splitlines()
        lines = all_lines[:print_lines]
        head = f'{fname} ➜ {len(text)} chars, {len(all_lines)} lines; first {len(lines)} lines:'
        dash = "-" * 100
        print(f'{dash}\n{head}\n{dash}')
        for line in lines:
            print(trunc(line))
        print(f'{dash}\nparse({day}) ➜ {len(entries)} entries:\n'
              f'{dash}\n{trunc(str(entries))}\n{dash}')
    return entries

def trunc(s: str, left=70, right=25, dots=' ... ') -> str: 
    """All of string s if it fits; else left and right ends of s with dots in the middle."""
    dots = ' ... '
    return s if len(s) <= left + right + len(dots) else s[:left] + dots + s[-right:]

In [3]:
Char = str # Intended as the type of a one-character string
Atom = Union[float, int, str]

def ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'-?[0-9]+', text))

def digits(text: str) -> Tuple[int]:
    """A tuple of all the digits in text (as ints 0–9), ignoring non-digit characters."""
    return mapt(int, re.findall(r'[0-9]', text))

def words(text: str) -> List[str]:
    """A list of all the alphabetic words in text, ignoring non-letters."""
    return re.findall(r'[a-zA-Z]+', text)

def atoms(text: str) -> Tuple[Atom]:
    """A tuple of all the atoms (numbers or symbol names) in text."""
    return mapt(atom, re.findall(r'[a-zA-Z_0-9.+-]+', text))

def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if round(x) == x else x
    except ValueError:
        return text
    
def mapt(fn, *args) -> tuple:
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

A few additional  utility functions that I have used in the past:

In [4]:
def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

class multimap(defaultdict):
    """A mapping of {key: [val1, val2, ...]}."""
    def __init__(self, pairs: Iterable[tuple], symmetric=False):
        """Given (key, val) pairs, return {key: [val, ...], ...}.
        If `symmetric` is True, treat (key, val) as (key, val) plus (val, key)."""
        self.default_factory = list
        for (key, val) in pairs:
            self[key].append(val)
            if symmetric:
                self[val].append(key)

def prod(numbers) -> float: # Will be math.prod in Python 3.8
    """The product formed by multiplying `numbers` together."""
    result = 1
    for x in numbers:
        result *= x
    return result

def total(counter: Counter) -> int: 
    """The sum of all the counts in a Counter."""
    return sum(counter.values())

def sign(x) -> int: return (0 if x == 0 else +1 if x > 0 else -1)

def transpose(matrix) -> list: return list(zip(*matrix))

def nothing(*args) -> None: return None

cat     = ''.join
flatten = chain.from_iterable
cache   = lru_cache(None)

Some past puzzles involve (x, y) points on a rectangular grid, so I'll define  `Point` and `Grid`:

In [5]:
Point = Tuple[int, int] # (x, y) points on a grid

neighbors4 = ((0, 1), (1, 0), (0, -1), (-1, 0))               
neighbors8 = ((1, 1), (1, -1), (-1, 1), (-1, -1)) + neighbors4

class Grid(dict):
    """A 2D grid, implemented as a mapping of {(x, y): cell_contents}."""
    def __init__(self, mapping=(), rows=(), neighbors=neighbors4):
        """Initialize with, e.g., either `mapping={(0, 0): 1, (1, 0): 2, ...}`,
        or `rows=[(1, 2, 3), (4, 5, 6)].
        `neighbors` is a collection of (dx, dy) deltas to neighboring points.`"""
        self.update(mapping if mapping else
                    {(x, y): val 
                     for y, row in enumerate(rows) 
                     for x, val in enumerate(row)})
        self.width  = max(x for x, y in self) + 1
        self.height = max(y for x, y in self) + 1
        self.deltas = neighbors
        
    def copy(self) -> Grid: return Grid(self, neighbors=self.deltas)
    
    def neighbors(self, point) -> List[Point]:
        """Points on the grid that neighbor `point`."""
        x, y = point
        return [(x+dx, y+dy) for (dx, dy) in self.deltas 
                if (x+dx, y+dy) in self]
    
    def to_rows(self) -> List[List[object]]:
        """The contents of the grid in a rectangular list of lists."""
        return [[self[x, y] for x in range(self.width)]
                for y in range(self.height)]

# [Day 1](https://adventofcode.com/2022/day/1): Calorie Counting


- **Input**: Each entry in the input is an calorie count of a single food. a group of calorie counts, delimited by empty space, represents food owned by a unique entity


In [6]:
# in1 = parse(2, int)
## returns a tuple. NOTE, tuple is just an immutable list (can have more than 2 elements)

tuple(map(int,[1,2,3]))

mapt(int, re.findall(r'-?[0-9]+', '13\n 14'))

# parse(2,int) ##the same results as above

# parse(1,atom) ##when blank lines are introduced, try atoms class


(13, 14)

In [7]:
# allcalories = parse(1, ints)
all_lines_list = parse(1,atom)
type(all_lines_list[2])
##we'll use their parser first for simple explaination, then write your own parser for text file
isblank = [x=='' for x in all_lines_list]

# somehow the last empty line doesn't make it into all_lines_list from parser, so number of blanks is off by 1
num_elves = sum(isblank)+1
num_elves

----------------------------------------------------------------------------------------------------
AOC2022/input1.txt ➜ 10415 chars, 2234 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
6750
6538
5292
4635
6855
4137
3840
----------------------------------------------------------------------------------------------------
parse(1) ➜ 2234 entries:
----------------------------------------------------------------------------------------------------
(6750, 6538, 5292, 4635, 6855, 4137, 3840, 4691, 1633, 6008, 2447, 144 ... , 5452, 7397, 7163, 4460)
----------------------------------------------------------------------------------------------------


239

- **Part 1**: which elf carries the largest group sum of calories?

In [8]:
# # for each elve, assign a group of calories to that entity
# list(range(1,10))

# for i in range(1,num_elves+1):
#     print(i)

# go through every line from all_lines_list, and keep appending to index 0 of a list until empty string, then iterate to next index
# list of lists struct where index of list is the elf's identifier
counter = 0
# w.o. the second brackets around None, it would be just a single list of elements (nonetype), cant append to element thats not a list
list_elves = [[] for elf_index in range(num_elves)] #creates list of lists, each element is a list of len 1 (they have to be unique to elf_index and not just references to 1 thing)
for entry in all_lines_list:
    if entry == '':
        counter+=1
        continue
    else:
        list_elves[counter].append(entry)


In [9]:
list_elves
# for each element of list, sum them all up.

# use map function
elf_calorie_holding = list(map(sum,list_elves))
# len(elf_calorie_holding)
max(elf_calorie_holding)

70374

In [10]:
##group them into groups per elf (separated by empty string)

# looked at input1.txt, and the number of empty lines is == to number of elves (theres an empty line after each grouping, including last)
# num_elves = 

#rewrite the parser so its understandable to you

# [Day 2](https://adventofcode.com/2022/day/2): Rock paper scissors


- **Input**: each row is other players input (ABC) and your projected response (XYZ)


In [11]:
# A for Rock, B for Paper, and C for Scissors
# response: X for Rock, Y for Paper, and Z for Scissors
# total score is the sum of your scores for each round. 
# The score for a single round is the score for the shape you selected (1 for Rock, 2 for Paper, and 3 for Scissors) plus the score for the outcome of the round (0 if you lost, 3 if the round was a draw, and 6 if you won).


## rock ties rock, rock loses to paper (YOU WIN), rock wins scissors (YOU LOSE), repeat
# paper ties paper, paper loses to scissors, paper wins rock...
# use a modulo with a tuple and if statement to calculate if you won or lost

list_matches = parse(2,atoms)
list_matches[0]


----------------------------------------------------------------------------------------------------
AOC2022/input2.txt ➜ 10000 chars, 2500 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
A Z
A Z
A Z
B Z
C X
A Z
A Z
----------------------------------------------------------------------------------------------------
parse(2) ➜ 2500 entries:
----------------------------------------------------------------------------------------------------
(('A', 'Z'), ('A', 'Z'), ('A', 'Z'), ('B', 'Z'), ('C', 'X'), ('A', 'Z' ... , ('B', 'Z'), ('A', 'Z'))
----------------------------------------------------------------------------------------------------


('A', 'Z')

In [12]:
from itertools import cycle

define_input = ['A', 'B', 'C']
define_response = ['X', 'Y', 'Z']



# next(pool)

In [13]:
from itertools import islice
import collections

def consume(iterator, n=None):
    "Advance the iterator n-steps ahead. If n is None, consume entirely."
    # Use functions that consume iterators at C speed.
    if n is None:
        # feed the entire iterator into a zero-length deque
        collections.deque(iterator, maxlen=0)
    else:
        # advance to the empty slice starting at position n
        next(islice(iterator, n, n), None)

In [14]:
# lets assume that input was B (paper) and your response was C (scissors)
pool = cycle(define_input)

consume(pool, 1) ##pool cycle is at paper
count=0
# next(pool)
while next(pool) != 'A': ##go until response
    count +=1
    print(count)
    # next(pool)
    # print(next(pool))
if count == 0:
    print('tie')
elif count ==1:
    print('win')
else: ##count ==2
    print('lose')

1
2
lose


In [77]:
# function evaluating win, loss, or tie in RPS


def score_game(moves: tuple):
    # display(moves[0], moves[1])
    define_input = ['A', 'B', 'C']
    define_response = ['X', 'Y', 'Z']
    RPS_dict = dict(zip(define_response, define_input))
    response_translated = RPS_dict[moves[1]]

    pool = cycle(define_input)
    consume(pool, define_input.index(moves[0])) ##iterate through cycle until you get to correct starting index
    count=0
    while next(pool) != response_translated: ##go until response
        count +=1
        if count >2:
            break
        # print(count)
    if count == 0:
        # print('tie')
        game_score = 3
    elif count ==1:
        # print('win')
        game_score = 6
    elif count ==2:
        # print('lose')
        game_score = 0
    else:
        print('inf loop')
    return game_score

    

# score_game(list_matches[0])



In [78]:
# define_input.index('A')
pool = cycle(define_input)
consume(pool, define_input.index('C'))
next(pool)

'C'

In [79]:
input = ('A', 'Z')
input[1]
score_game(input)

# if input == 'A', and response = 'Y', you want to find out how many iterations ahead the response is from input

0

In [80]:
# score for response you selected. 1 for Rock, 2 for Paper, and 3 for Scissors
def score_shape(response):
    if response == 'X':
        score =1
    elif response == 'Y':
        score =2
    else:
        score =3
    return score

In [83]:
your_total_score = 0
for match in list_matches:
    # score_game(match)
    # print(score_shape(match[1]))

    round_score = (score_game(match)+score_shape(match[1]))
    your_total_score += round_score

your_total_score

12535

## [Day 2 part 2](https://adventofcode.com/2022/day/2): Rock paper scissors


- **Input**: each row is other players input (ABC) and the result of the round (X means you lose, Y means you tie, and Z means you win)

In [74]:
parse(2,atoms)

def score_game2(moves: tuple):
    define_input = ['A', 'B', 'C']
    define_result_round = ['X', 'Y', 'Z']

    pool = cycle(define_input)
    ##behavior of consume is such that the following time you call next(pool will give you whatever move moves[0] is)
    consume(pool, define_input.index(moves[0])) ##iterate through cycle until you get to correct starting index for first player
    
    if moves[1] == 'Y':
        # print('tie')
        game_score = 3
    elif moves[1] == 'Z':
        # print('win')
        game_score = 6
    elif moves[1] == 'X':
        # print('lose')
        game_score = 0
    else:
        print('error in result given')
    return game_score

def determine_response_move(moves: tuple):
    define_input = ['A', 'B', 'C']
    define_result_round = ['X', 'Y', 'Z']
    pool = cycle(define_input)
    consume(pool, define_input.index(moves[0])) ##cycle through pool until the next(pool) will give you moves[0]
    
    if moves[1] == 'Y':
        # print('tie')
        num_steps_to_advance=0
    elif moves[1] == 'Z':
        # print('win')
        num_steps_to_advance=1
    elif moves[1] == 'X':
        # print('lose')
        num_steps_to_advance=2
    else:
        print('error in result given')
        num_steps_to_advance=None

    for i in range(num_steps_to_advance+1): ##neec +1 to account for the fact taht moves[0] needs to get passed first
        response_move = next(pool)

    return response_move ##given in ABC format

def score_shape2(response):
    if response == 'A':
        score =1
    elif response == 'B':
        score =2
    else:
        score =3 ##response is C
    return score

def score_total2(list_of_matches):
    your_total_score=0
    for match in list_matches:
        round_score = score_game2(match)+score_shape2(determine_response_move(match))
        your_total_score += round_score
    return your_total_score

----------------------------------------------------------------------------------------------------
AOC2022/input2.txt ➜ 10000 chars, 2500 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
A Z
A Z
A Z
B Z
C X
A Z
A Z
----------------------------------------------------------------------------------------------------
parse(2) ➜ 2500 entries:
----------------------------------------------------------------------------------------------------
(('A', 'Z'), ('A', 'Z'), ('A', 'Z'), ('B', 'Z'), ('C', 'X'), ('A', 'Z' ... , ('B', 'Z'), ('A', 'Z'))
----------------------------------------------------------------------------------------------------


In [73]:
score_shape2(determine_response_move(list_matches[7]))
score_game2(list_matches[7])

your_total_score2 = 0
for match in list_matches:
    # score_game(match)
    # print(score_shape(match[1]))

    round_score = score_game2(match)+score_shape2(determine_response_move(match))
    your_total_score2 += round_score
your_total_score2

15457

In [76]:
score_total2(list_matches) ##answer to day 2 part 2

15457